In [13]:
import os
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm
tqdm.pandas()

## BEFORE YOU START... 
#1. install libraries
#2. install chromedriver >> reference :: (https://beomi.github.io/gb-crawling/posts/2017-02-27-HowToMakeWebCrawler-With-Selenium.html)
#3. make a new folder for save book covers

import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
# !pip install selenium
from selenium import webdriver
import urllib


import warnings
warnings.filterwarnings("ignore")

In [108]:
def get_book_metadata(start, end, book_id_list, webdriver_path, image_save_path):
    meta_df = pd.DataFrame()
    
    driver = webdriver.Chrome(webdriver_path) 
    driver.implicitly_wait(3)

    for i in range(start, end):
        print(i, end = ' ')
    
        book_id = book_id_list[i]
        time.sleep(1)
        
        try:
            time.sleep(1)
            url = "https://lib.skku.edu/#/search/total/si?all=1%7Ck%7Ca%7C{}".format(book_id)
            driver.get(url)
    
            result = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[2]/div[2]/div[3]/div/ik-total-search[1]/div/div[2]/div/div[2]/ul/li[1]/a[2]')
            result.click()
    
            driver.switch_to.window(driver.window_handles[1])
    
            #open the detailed info 
            try:
                driver.find_element_by_css_selector('#btn-biblio-more-open').click()
            except: 
                pass

    
            #image retreival 
            book_cover = driver.find_elements_by_css_selector('div.ikc-bookcover')
            img_url = re.findall('(?<=src=").+?(?=")', book_cover[0].get_attribute('outerHTML'))[0]
    
            try:
                img_name = image_save_path + '/' + book_id + '.jpg'
                urllib.request.urlretrieve(img_url,img_name)
                img = 'Y'
        
            except:
                img = 'N'
            
  
            ##book title 
            book_title = driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(1) > span').text
            title = book_title.split('/')[0]
            
            pub_year = np.nan
            page = np.nan
            isbn = np.nan
    
            for i in range(3,9):
                try:
                    text = driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(' + str(i) + ') > label').text
                    
                    ##book pub year 
                    if text == '발행사항':
                        book_pub= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(' + str(i) + ') > span > span').text
                        pubyear= book_pub.split(',')[-1]
        
                    ## book page 
                    elif text == '형태사항':
                        book_page= driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(' + str(i) + ') > span > span').text
                        page = book_page.split(' p.')[0]

                    ## isbn
                    elif text == 'ISBN':
                        isbn = driver.find_element_by_css_selector('body > div.ikc-pyxis-wrap > div.ikc-container-wrap > div.ikc-container > div > div.ikc-main.kku-search > div.kku-main > div:nth-child(2) > div > div.ikc-search-detail-main > div > div.ikc-biblio-detail > div.ikc-biblio-info > ul > li:nth-child(' + str(i) + ') > span > span').text
                        
                    else: 
                        pass
                    
                except:
                    break
    
            meta_df = meta_df.append({"book_id" : book_id, "isbn" : isbn, "subject" : title, 
                                      "pub_year" : pubyear, "page" : page, 'img' : img}, ignore_index = True)
            
            driver.close()
            driver.switch_to_window(driver.window_handles[0])

        
        except:
            meta_df = meta_df.append({"book_id" : book_id, "isbn" : 'notfound', 'subject' : 'notfound',
                                      'pub_year' : 'notfound', 'page': 'notfound', 'img':'notfound'}, ignore_index = True)
            
    return meta_df

In [ ]:
def clean_title(x):
    try:
        x = x.replace('&nbsp;', ' ')
    except:
        pass

    try: 
        x = x.split('=')
        return x[0]
    except:
        return x


def clean_page(x):
    if x == "notfound":
        return x
    else: 
        try: 
            float(x)
            return float(x)
        
        except: 
            if 'p' in x:
                new_x = x.split('p')
                try:
                    return float(new_x[0].split(' ')[-1])
                except:
                    try:
                        return float(new_x[1].split(' ')[0])
                    except:
                        try:
                            return float(x.split(' p')[0].split(' ')[0])
                        except:
                            try: 
                                return float(x.split(' p')[0].split(' ')[1])
                            except:
                                return np.nan
                
        
            elif ('cm' in x) or ('책' in x) or ('冊' in x) : 
                return np.nan
            else:
                if ',' in x:
                    new_x = x.split(',')
                    if 'p' in x:
                        try:
                            return float(x.split('p')[0])
                        except:
                            return np.nan
                    
                
                    elif len(new_x) == 2:
                        try:
                            return float(new_x[0])
                        except:
                            try:
                                return float(new_x[1])
                            except:
                                if '.' in new_x[1]:
                                    try: 
                                        return float(new_x[1].split('.')[0])
                                    except:
                                        return np.nan                                
                                elif ';' in new_x[1]:
                                    try:
                                        return float(new_x[1].split(';')[0])
                                    except:
                                        return np.nan
                                else:
                                    return np.nan
               
                    elif len(new_x) == 3:
                        try: 
                            return float(new_x[1])
                        except:
                            try:
                                return float(new_x[2])
                            except:
                                try:
                                    return float(new_x[0])
                                except:
                                    return np.nan
                
               
                elif ';' in x:
                    try:
                        return float(x.split(';')[-1])
                    except:
                        if 'p' in x:
                            try:
                                return float(x.split('p')[0])
                            except:
                                return np.nan
                        else:
                            return np.nan

def clean_pub_year(x):
    try: 
        return float(x)
    
    except:
        r = list(x)        
        s=''
        for i in r:
            if i.isdigit():
                s+=str(i)
            else:
                pass
            
        #for years which have duration (first year)
        s = s[:4]
        
        #for misinfo 
        if s == '':
            if x == 'notfound':
                return np.nan
            else: 
                return np.nan
        else:
            return float(s)


def clean_isbn(row):
    x = row['isbn']
    if pd.isna(x):
        x, remark_x = np.nan, np.nan
        
    else:
        if '(' in x:
            new_x = x.split('(')
            x = new_x[0]
            remark_x = new_x[1].strip(')')
    
        elif '[' in x:
            new_x = x.split('[')
            x = new_x[0]
            remark_x = new_x[1].strip(']')
    
        else:
            remark_x = np.nan
    
    return [x, remark_x]


def split_copy_n_vol(row):
    x = row['book_ddc']
    if pd.isna(x):
        x, copy, vol = np.nan, np.nan, np.nan
    else:
        x = x.replace('[', '').replace(']', '')
        try:
            copy = x.split('c.')[1]
            x = x.split('c.')[0]
        except:
            copy = np.nan
        
        try:
            vol = x.split('v.')[1]
            x = x.split('v.')[0]
        except:
            vol = np.nan
    
    return [x, copy, vol]

def get_genre_from_ddc(row):
    x = row['short_ddc']
    
    if x[0].isdigit():
        gen_num = int(x[0])
    else:
        gen_num = int(x[2])
        
    genre_dicts = {0: 'Computer science, information and general works',
                   1:'Philosophy and psychology',
                   2: 'Religion', 
                   3: 'Social sciences',
                   4: 'Language',
                   5: 'Science',
                   6:'Technology',
                   7:'Arts and recreation',
                   8:'Literature',
                   9:'History and geography'}
    
    gen_words = genre_dicts[gen_num]
    
    return gen_words


def make_unique_isbn(row):
    if (pd.isna(row['isbn']) == True) or (pd.isna(row['volume']) == True):
        return row['isbn']
    else:
        return str(row['isbn']) + '_' + str(row['volume'])
    

def clean_metadata(df, org_df, save_mode = False):
    # cleaning subject, page, publish year
    df['subject'] = df['subject'].apply(lambda x : clean_title(x))
    df['page'] = df['page'].apply(lambda x : clean_page(x))
    df['pub_year'] = df['pub_year'].apply(lambda x: clean_pub_year(x))
    
    print('>>> CLEANED Title, Page, Publish year')
    
    # cleaning isbn : separating remarks from isbn (13 digits)
    temp1 = df.apply(lambda x : pd.Series(clean_isbn(x), index = ['isbn', 'isbn_remark']), axis = 1, result_type='expand')
    
    print('>>> CLEANED ISBN')
    
    # concatenating other features with isbn and isbn remarks
    df = pd.concat([df[['book_id', 'img', 'page', 'pub_year', 'subject']], temp1], axis = 1)

    # merge with book df and book metadata df
    total_df = pd.merge(org_df, df[['book_id', 'img', 'page', 'pub_year', 'isbn', 'isbn_remark']], on = 'book_id', how = 'left')
    
    # separating ddc string as short ddc, copy(c.1) and volume(v.1)
    temp2 = total_df.apply(lambda x : pd.Series(split_copy_n_vol(x), index = ['short_ddc', 'copy', 'volume']), axis = 1, result_type = 'expand')
    
    print('>>> SPLIT DDC into Short-ddc, Copy and Volume')
    
    # concatenating other features with temp2
    total_df = pd.concat([total_df, temp2], axis = 1)
    
    # make unique isbn
    total_df['unique_isbn'] = total_df.apply(lambda x: make_unique_isbn(x), axis = 1)
    
    # get genre from ddc
    total_df['genre'] = total_df.progress_apply(lambda x: get_genre(x), axis = 1)
    
    print('>>> MADE Unique ISBN')
    
    # drop na
    final_df = total_df[['book_id', 'title', 'short_ddc', 'unique_isbn', 'pub_year', 'page', 'img']].dropna()
    final_df = final_df[final_df['img'] == 'Y']
    
    print(">>> DROP NA and IMG == N")
    
    # arrange
    final_df.drop_duplicates('label', inplace=True)
    final_df.sort_values('label', inplace=True)
    final_df.set_index('label', inplace=True)
    final_df.reset_index(inplace=True)
    
    if save_mode == True:
        csv_save_name_total = 'book_data_total_' + str(start) + '_' + str(end) + '.csv'
        csv_save_name_final = 'book_data_final_' + str(start) + '_' + str(end) + '.csv'
        
        total_df.to_csv(csv_save_name_total)
        final_df.to_csv(csv_save_name_final)
        
    return total_df, final_df

In [ ]:
start, end = 0, 100
webdriver_path = '../chromedriver'
image_save_path = '../image'

if __name__ == '__main__':
    book_df = pd.read_csv('../book.csv', index_col = 0)
    book_ids = book_df['book_id'].unique()
    meta_df = get_book_metadata(start, end, book_ids, webdriver_path, image_save_path)
    total_df, final_df = clean_metadata(meta_df, book_df)